In [1]:
from torchvision.datasets import ImageFolder
from torchvision import transforms as tt
from torch.utils.data import DataLoader, random_split, ConcatDataset
import torch
from torch import nn
from torch import optim
from digit_recognizer import DigitRecognizer

In [2]:
folder_path = "/Users/osiprovin/Desktop/ml:dl/CV/Magic math notebook/Dataset"

In [3]:
transforms = tt.Compose([
    tt.Resize((28, 28)),
    tt.Grayscale(),
    tt.ToTensor()
])

In [4]:
folder = ImageFolder(folder_path, transforms)

train_size = int(0.8 * len(folder))
test_size = len(folder) - train_size

# Split the dataset
train_folder, test_folder = random_split(folder, [train_size, test_size])

# Create DataLoaders
batch_size = 4

train_loader = DataLoader(train_folder, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_folder, batch_size=batch_size, shuffle=True)

In [11]:
device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")

digit_recognizer = DigitRecognizer().to(device)

criterion = nn.CrossEntropyLoss().to(device)

optimizer = optim.Adam(digit_recognizer.parameters(), lr=0.0002)

num_epochs = 5

In [12]:
train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []

for epoch in range(num_epochs):
    
    # Train
    digit_recognizer.train()
    train_loss = 0.0
    correct_train = 0
    
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        inputs = inputs.to(device)
        labels = labels.to(device)
        proba = digit_recognizer(inputs)
        loss = criterion(proba, labels)
        train_loss += loss.item()
        loss.backward()
        optimizer.step()
        preds = digit_recognizer.proba_to_predict(proba)
        correct_train += (preds == labels).sum().item()
        
    
    train_loss /= len(train_folder)
    train_losses.append(train_loss)
    train_accuracy = 100 * correct_train / len(train_folder)
    train_accuracies.append(train_accuracy)
    
    
    # Validationx
    digit_recognizer.eval() 
    val_loss = 0.0
    correct_val = 0
    
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            proba = digit_recognizer(inputs)
            loss = criterion(proba, labels)
            val_loss += loss.item()
            preds = digit_recognizer.proba_to_predict(proba)
            correct_val += (preds == labels).sum().item()
    
    val_loss /= len(test_folder)
    val_losses.append(val_loss)
    val_accuracy = 100 * correct_val / len(test_folder)
    val_accuracies.append(val_accuracy)
    
    print(f"Epoch {epoch+1}, Training Loss: {round(train_loss, 5)}, Training Accuracy: {round(train_accuracy, 2)}%, Validation Loss: {round(val_loss, 5)}, Validation Accuracy: {round(val_accuracy, 2)}%")
    break

digit_recognizer = digit_recognizer.cpu()
torch.save(digit_recognizer.state_dict(), 'digit_recognizer.pth')

Epoch 1, Training Loss: 0.05804, Training Accuracy: 93.12%, Validation Loss: 0.01248, Validation Accuracy: 98.8%
Epoch 2, Training Loss: 0.01509, Training Accuracy: 98.35%, Validation Loss: 0.00824, Validation Accuracy: 99.28%
Epoch 3, Training Loss: 0.01031, Training Accuracy: 98.89%, Validation Loss: 0.00601, Validation Accuracy: 99.51%
Epoch 4, Training Loss: 0.00829, Training Accuracy: 99.16%, Validation Loss: 0.00535, Validation Accuracy: 99.57%
Epoch 5, Training Loss: 0.00703, Training Accuracy: 99.28%, Validation Loss: 0.00416, Validation Accuracy: 99.7%
